In [ ]:
# sqlite and pandas required
# Jiyang Li 2025-10-08


import sqlite3
import pandas as pd
df = pd.read_csv('../../data/PoliticalDiscussion_comments_sample.csv')
conn = sqlite3.connect('project_test.db')
df.to_sql('comment_table', conn, if_exists='replace', index=False)
post_number = pd.read_sql_query("select count(distinct parent_id) from comment_table ", conn)
print(post_number)



/tmp/ipykernel_10447/717437559.py:7: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/PoliticalDiscussion_comments_sample.csv')


   count(distinct parent_id)
0                      48522


In [4]:
comment_number = pd.read_sql_query("select count(distinct comment_id) from comment_table ", conn)
print(comment_number)

   count(distinct comment_id)
0                       66942


In [ ]:
# data structure
df.head(5)

,kind,subreddit,submission_id,comment_id,parent_id,is_top_level,body,author,score,created_utc,author_flair_text,author_flair_type,author_flair_template_id,author_flair_richtext
0,comment,PoliticalDiscussion,litsrx,gn9kh4p,t1_gn5o9dz,False,Pedantic but it does have kne exception. India...,Mist_Rising,27,1.613242e+09,NaN,text,NaN,[]
1,comment,PoliticalDiscussion,fjqvg5,fkp06ve,t3_fjqvg5,True,Tammy Baldwin or Elizabeth Warren are who I wa...,nantesorkestar,136,1.584399e+09,NaN,text,NaN,[]
2,comment,PoliticalDiscussion,w7yls1,ihngruw,t1_ihn2l7x,False,https://www.usatoday.com/story/news/politics/2...,lvlint67,6,1.658795e+09,NaN,text,NaN,[]
3,comment,PoliticalDiscussion,13zqss9,jmuzwk2,t1_jmuz8v0,False,"You bet, here:\n\nhttps://www.bbc.com/news/wor...",sbdude42,9,1.685885e+09,NaN,text,NaN,[]
4,comment,PoliticalDiscussion,13u8sip,jm0aig8,t1_jm09faz,False,I’ll tell you before I (involuntarily) lost my...,2057Champs__,6,1.685318e+09,NaN,text,NaN,[]


In [3]:
df=df.drop(columns=['author_flair_text','author_flair_type','author_flair_template_id','author_flair_richtext'])
df.head(5)

,kind,subreddit,submission_id,comment_id,parent_id,is_top_level,body,author,score,created_utc
0,comment,PoliticalDiscussion,litsrx,gn9kh4p,t1_gn5o9dz,False,Pedantic but it does have kne exception. India...,Mist_Rising,27,1.613242e+09
1,comment,PoliticalDiscussion,fjqvg5,fkp06ve,t3_fjqvg5,True,Tammy Baldwin or Elizabeth Warren are who I wa...,nantesorkestar,136,1.584399e+09
2,comment,PoliticalDiscussion,w7yls1,ihngruw,t1_ihn2l7x,False,https://www.usatoday.com/story/news/politics/2...,lvlint67,6,1.658795e+09
3,comment,PoliticalDiscussion,13zqss9,jmuzwk2,t1_jmuz8v0,False,"You bet, here:\n\nhttps://www.bbc.com/news/wor...",sbdude42,9,1.685885e+09
4,comment,PoliticalDiscussion,13u8sip,jm0aig8,t1_jm09faz,False,I’ll tell you before I (involuntarily) lost my...,2057Champs__,6,1.685318e+09


In [6]:
#should use pandas not sql :(
average_len = pd.read_sql_query("select sum(length(body))/count(*) from comment_table limit 5 ", conn)
print(average_len)

   sum(length(body))/count(*)
0                         368


In [9]:
# count by year
df['year'] = df['created_time'].dt.year
year_counts = df['year'].value_counts().sort_index()
print(year_counts)

year
2012       35
2013       58
2014       30
2015      117
2016     4992
2017     2226
2018     1509
2019     1296
2020     5906
2021     7627
2022    22189
2023    20957
Name: count, dtype: int64


In [ ]:
#check whether there are deleted comments
cursor = conn.cursor()
pattern = "%removed%" #avoid %r in text
cursor.execute("SELECT * FROM comment_table WHERE length(body)<=20 and body LIKE ?", (pattern,))
rows = cursor.fetchall()
print(rows)

[]


In [ ]:
cursor = conn.cursor()
pattern = "%deleted%" #avoid %r in text
cursor.execute("SELECT * FROM comment_table WHERE length(body)<=20 and body LIKE ?", (pattern,))
rows = cursor.fetchall()
print(rows)

[]


In [1]:
#merge submissions and comments
# Plz add the time transformation code to the merged data frame
# !!! data structure is a little bit messy here, try to clean it and drop some useless columns
import pandas as pd

comments = pd.read_csv("../data/PoliticalDiscussion_comments_sample.csv")
submissions = pd.read_csv("../data/PoliticalDiscussion_submissions_sample.csv")



C:\Users\16082\AppData\Local\Temp\ipykernel_20784\902415326.py:6: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  comments = pd.read_csv("../data/PoliticalDiscussion_comments_sample.csv")


In [2]:
comments=comments.drop(columns=['author_flair_text','author_flair_type','author_flair_template_id','author_flair_richtext'])

In [3]:
# change created_utc to time
from datetime import datetime, timezone, timedelta


comments['created_time'] = comments['created_utc'].apply(
    lambda x: datetime.fromtimestamp(x, tz=timezone.utc)
)

print(comments.head())


      kind            subreddit submission_id comment_id   parent_id  \
0  comment  PoliticalDiscussion        litsrx    gn9kh4p  t1_gn5o9dz   
1  comment  PoliticalDiscussion        fjqvg5    fkp06ve   t3_fjqvg5   
2  comment  PoliticalDiscussion        w7yls1    ihngruw  t1_ihn2l7x   
3  comment  PoliticalDiscussion       13zqss9    jmuzwk2  t1_jmuz8v0   
4  comment  PoliticalDiscussion       13u8sip    jm0aig8  t1_jm09faz   

   is_top_level                                               body  \
0         False  Pedantic but it does have kne exception. India...   
1          True  Tammy Baldwin or Elizabeth Warren are who I wa...   
2         False  https://www.usatoday.com/story/news/politics/2...   
3         False  You bet, here:\n\nhttps://www.bbc.com/news/wor...   
4         False  I’ll tell you before I (involuntarily) lost my...   

           author  score   created_utc              created_time  
0     Mist_Rising     27  1.613242e+09 2021-02-13 18:52:24+00:00  
1  nantesork

In [4]:
# earliest and latest
earliest = comments['created_time'].min()
latest = comments['created_time'].max()
print("earliest time = ", earliest)
print("latest time = ", latest)

earliest time =  2012-01-25 15:55:43+00:00
latest time =  2023-12-30 13:28:46+00:00


In [5]:
# count by year
comments['year'] = comments['created_time'].dt.year
year_counts = comments['year'].value_counts().sort_index()
print(year_counts)

year
2012       35
2013       58
2014       30
2015      117
2016     4992
2017     2226
2018     1509
2019     1296
2020     5906
2021     7627
2022    22189
2023    20957
Name: count, dtype: int64


In [6]:
comments["parent_clean_id"] = comments["parent_id"].str.replace(r"t\d+_", "", regex=True)
merged = pd.merge(
    submissions, comments,
    how="outer",
    left_on="submission_id",
    right_on="parent_clean_id",
    indicator=True
)

In [10]:
merged=merged.drop(columns=['link_flair_text','link_flair_type','link_flair_template_id','link_flair_richtext'])

In [ ]:
merged.to_csv("../../data/joined_data.csv", index=False)

In [14]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70494 entries, 0 to 70493
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype              
---  ------           --------------  -----              
 0   kind_x           17668 non-null  object             
 1   subreddit_x      17668 non-null  object             
 2   submission_id_x  17668 non-null  object             
 3   title            17668 non-null  object             
 4   selftext         17668 non-null  object             
 5   author_x         17668 non-null  object             
 6   score_x          17668 non-null  float64            
 7   created_utc_x    17668 non-null  float64            
 8   kind_y           66942 non-null  object             
 9   subreddit_y      66942 non-null  object             
 10  submission_id_y  66942 non-null  object             
 11  comment_id       66942 non-null  object             
 12  parent_id        66942 non-null  object             
 13  is_top_level    